In [6]:
yoi = [2020]
toi = ['15RTN', '15RTP', '14RNS']
root_path = "C:/Users/mcecil/CGA/CDL/"
spath = root_path + f"CDL_HLS_dataframe{yoi[0]}.csv"
url_file = root_path + f'CDL_HLS_dataframe{yoi[0]}.csv'
index_file = root_path + f'mask_index_{yoi[0]}.csv'
kml_file = root_path + 'sentinel_tile_grid.kml'
geojson_file = root_path + 'aoi.geojson'  ## chip file, lat-long
geojson_rpj_file = root_path + 'aoi_rpj.geojson'
hdf_dir = root_path + 'hdf/'
tiff_dir = root_path + 'tif/'
mask_dir = root_path + 'mask/'

tile_src_path = root_path + "tile_src.csv"

# New tile edge length (in pixels)
npixels = 512

# Directory containing scenes
scdir = root_path + 'scenes/'

# Directory containing masks
madir = root_path +  'masks/'

# Location to save new files
svdir = root_path +  'subsetted_USSE/'


In [4]:

# Switch plotting backend
import matplotlib
matplotlib.use('agg')

# Import required modules
#import cartopy.crs as ccrs
import os
from datetime import datetime
import pandas as pd
import geopandas
#import pyhdf
import rasterio
import rasterio.mask
import matplotlib.pyplot as pp
import nasa_hls
import numpy as np
import time
import fiona
from pathlib import Path
import time
import urllib.request as urlreq
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
# Switch plotting backend
import matplotlib
matplotlib.use('agg')


### Import modules
from glob import glob
import numpy as np
import matplotlib.pyplot as pp
import rasterio

In [30]:
from rasterio.windows import Window


### Locate mask files and loop over them
masks = sorted(glob(madir+'*.tif'))
#print(madir+'*.tif')

for mask in masks[0:2]:
    print(mask)
    # Read data
    src = rasterio.open(mask)
    data = src.read(1)
    
    # Skip if no valid pixels
    if (np.sum(data == 1) == 0):
        continue
    
    # Locate burn scar center point
    xinds = np.arange(data.shape[0])
    yinds = np.arange(data.shape[1])
    print(xinds)
    print(yinds)
    xg, yg = np.meshgrid(xinds, yinds)
    xc = np.floor(np.mean(xg[data==1]))
    yc = np.floor(np.mean(yg[data==1]))
    print(xc)
    print(yc)
    # Compute edges of new box
    x1 = int(xc-npixels//2)
    x2 = int(xc+npixels//2)
    y1 = int(yc-npixels//2)
    y2 = int(yc+npixels//2)
    
    # Check if new inds are out of bounds
    if (x2 > data.shape[0]+1):
        shift = x2-(data.shape[0]+1)
        x2 = x2 - shift
        x1 = x1 - shift
        
    if (x1 < 0):
        shift = abs(x1)+1
        x1 = x1 + shift
        x2 = x2 + shift
        
    if (y2 > data.shape[1]+1):
        shift = y2-(data.shape[1]+1)
        y2 = y2 - shift
        y1 = y1 - shift
        
    if (y1 < 0):
        shift = abs(y1)+1
        y1 = y1 + shift
        y2 = y2 + shift

    # Subset the mask
    subset_data = np.expand_dims(data[y1:y2, x1:x2], axis=0)

    # Save new mask
    print(data[x1:x2, y1:y2].shape)
    print(x1,x2)
    print(y1,y2)
    fig, (ax1, ax2) = pp.subplots(ncols=2)
    ax1.imshow(data)
    ax1.set_title('Whole Scene Mask', fontsize=14, fontweight='roman')
    ax2.imshow(subset_data[0,:,:])
    ax2.set_title('Subsetted', fontsize=14, fontweight='roman')
    jpg_name = mask.split("\\")[-1].replace("tif", "jpg")
    pp.savefig(f'{svdir}mask_plots/{jpg_name}')
    pp.close()
        
    # Save the new mask
    out_meta = src.meta
    print(out_meta)
    out_meta.update({'driver':'GTiff', 'height':subset_data.shape[1],
        'width':subset_data.shape[2]})
    file_name = mask.split("\\")[-1]
#     with rasterio.open(f'{svdir}/masks/subsetted_{npixels}x{npixels}_{file_name}', 'w', **out_meta) as dest:
#         dest.write(subset_data)
    print(x1, y1, x2-x1, y2-y1)
    window = Window(x1, y1, x2-x1, y2-y1)
    print(window)
    with rasterio.open(
        f'{svdir}/masks/subsetted_{npixels}x{npixels}_test_{file_name}', 'w',
        **out_meta) as dst:
        dst.write(data, window= window, indexes=1)
    # Grab matching scene files
#     name = mask.split("/")[-1].replace(".mask.tif", "")
#     scene_dir = f'{scdir}/{name}'
    
#     ### Comment out unwanted bands
#     #src1 = rasterio.open(f'{scene_dir}/{name}__Coastal_Aerosol.tif')
#     src2 = rasterio.open(f'{scene_dir}/{name}__Blue.tif')
#     src3 = rasterio.open(f'{scene_dir}/{name}__Green.tif')
#     src4 = rasterio.open(f'{scene_dir}/{name}__Red.tif')
#     #src5 = rasterio.open(f'{scene_dir}/{name}__Red_Edge1.tif')
#     #src6 = rasterio.open(f'{scene_dir}/{name}__Red_Edge2.tif')
#     #src7 = rasterio.open(f'{scene_dir}/{name}__Red_Edge3.tif')
#     #src8 = rasterio.open(f'{scene_dir}/{name}__NIR_Broad.tif')
#     src8A = rasterio.open(f'{scene_dir}/{name}__NIR_Narrow.tif')
#     #src11 = rasterio.open(f'{scene_dir}/{name}__SWIR1.tif')
#     #src12 = rasterio.open(f'{scene_dir}/{name}__SWIR2.tif')
#     #src9 = rasterio.open(f'{scene_dir}/{name}__Water_Vapor.tif')
#     #src10 = rasterio.open(f'{scene_dir}/{name}__Cirrus.tif')
    
#     bands = []
#     for src in (src2, src3, src4, src8A): # Set the tuple to match desired bands
#         bands.append(src.read(1)[y1:y2, x1:x2])
#     bands = np.array(bands)
    
#     # Write to new tiff
#     out_meta.update({'driver':'GTiff', 'height':bands.shape[1],
#         'width':bands.shape[2], 'count':bands.shape[0], 'dtype':bands.dtype})
#     save_name = f'{svdir}/scenes/subsetted_{npixels}x{npixels}_{name}_merged.tif'
#     with rasterio.open(save_name, 'w', **out_meta) as dest:
#         dest.write(bands)

C:/Users/mcecil/CGA/CDL/masks\HLS.S30.T14RNS.2020002.v1.4.mask.tif
C:/Users/mcecil/CGA/CDL/masks\HLS.S30.T14RNS.2020005.v1.4.mask.tif
[   0    1    2 ... 3657 3658 3659]
[   0    1    2 ... 3657 3658 3659]
1698.0
1625.0
(512, 512)
1442 1954
1369 1881
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -1000.0, 'width': 3660, 'height': 3660, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unnamed",GEOGCS["Unknown datum based upon the WGS 84 ellipsoid",DATUM["Not_specified_based_on_WGS_84_spheroid",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-99],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(30.0, 0.0, 499980.0,
       0.0, -30.0, 3200040.0)}


RasterioIOError: Read or write failed. C:/Users/mcecil/CGA/CDL/subsetted_USSE/masks/subsetted_512x512_test_HLS.S30.T14RNS.2020005.v1.4.mask.tif: Access window out of range in RasterIO().  Requested (1442,1369) of size 512x512 on raster of 512x512.

In [21]:
from osgeo import gdal

raster_bounds = src.bounds
print(raster_bounds)
upper_left_x = 699934.584491
upper_left_y = 6169364.0093
lower_right_x = 700160.946739
lower_right_y = 6168703.00544
window = (upper_left_x,upper_left_y,lower_right_x,lower_right_y)

gdal.Translate('output_crop_raster.tif', 'input_raster.tif', projWin = window)

BoundingBox(left=499980.0, bottom=3090240.0, right=609780.0, top=3200040.0)


ValueError: Received a NULL pointer.